In [1]:
import pandas as pd
import requests
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# =========== 🚀 获取BTC历史价格数据 =============
def get_btc_data(days=365):
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "daily"
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    prices = data['prices']
    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    return df[['Date', 'Price']]

df = get_btc_data(365)

# =========== 🚀 Dash 应用 =============
app = Dash(__name__)

app.layout = html.Div([
    html.H1("比特币 (BTC) 最近一年历史价格滑动图", style={"textAlign": "center"}),
    dcc.Slider(
        id='price-slider',
        min=0,
        max=len(df)-1,
        step=1,
        value=len(df)-1,
        marks={i: df.iloc[i]["Date"].strftime("%m-%d") for i in range(0, len(df), 30)},
        tooltip={"placement": "bottom", "always_visible": False}
    ),
    dcc.Graph(id='btc-price-graph')
])

@app.callback(
    Output('btc-price-graph', 'figure'),
    Input('price-slider', 'value')
)
def update_graph(selected_index):
    filtered_df = df.iloc[:selected_index+1]
    fig = px.line(filtered_df, x='Date', y='Price', title='BTC 最近一年价格走势')
    fig.update_traces(line_color='orange')
    fig.update_layout(transition_duration=300)
    return fig

if __name__ == '__main__':
    app.run(debug=True)
